# Weather and Electricity Data Preprocessing Notebook


### Importing libraries

In [1]:

import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler




### Data formating

In [2]:

def format_climate(file_path):
    df = pd.read_csv(file_path)
    df.replace('-', pd.NA, inplace=True)
    #df.dropna(inplace=True)

    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    df = df.drop(columns='Station')

    df['Total Rainfall Mm'] = pd.to_numeric(df['Total Rainfall Mm'], errors='coerce')
    df['Mean Air Temperature in Celsius'] = pd.to_numeric(df['Mean Air Temperature in Celsius'], errors='coerce')
    df['Mean Vapour Pressure Hpa'] = pd.to_numeric(df['Mean Vapour Pressure Hpa'], errors='coerce')
    df['Mean Wind Speed M/Sec'] = pd.to_numeric(df['Mean Wind Speed M/Sec'], errors='coerce')
    df['Days Of Wind Gust >= 24 Knots in Day'] = pd.to_numeric(df['Days Of Wind Gust >= 24 Knots in Day'], errors='coerce')
    df['Standard Deviation Of Daily Mean Temperature in Celsius'] = pd.to_numeric(df['Standard Deviation Of Daily Mean Temperature in Celsius'], errors='coerce')

    df_grouped = df.groupby('Date').mean().reset_index()

    return df_grouped

In [3]:
def format_eletricity_data(file_path, business = False):
    df = pd.read_csv(file_path)
    
    df['Date'] = pd.to_datetime(df['Period start'], format='%d/%m/%y')
    #df.dropna(inplace=True)
    #df.reset_index(drop=True, inplace=True)


    if business:
        df = df.drop(columns=[
            'Regional demand (GWh)',
            'Proportion of regional demand (%)', 
            'National demand (GWh)',
            'Proportion of national demand (%)'])
        return df
    
    df = df.drop(columns=[
        'Est. Total demand (GWh)',
        'Est. proportion of regional demand',
        'Est. proportion of national demand', 'Average consumption (kWh)',
        'Average daily consumption (kWh)', '5th percentile (kWh)',
        '25th percentile (kWh)', '50th percentile (kWh)',
        '75th percentile (kWh)', '95th percentile (kWh)', 'Coverage %'])

    return df
    

In [4]:
def read_climate_list(file_list, file_path):
    data_list = []
    zone_names = ['UNI', 'LSI', 'USI', 'LNI', 'CNI']

    for i in file_list:
        data_list.append(format_climate(file_path+ i))
    
    for i in range(len(data_list)):
        data_list[i]['Region ID'] = zone_names[i]
        
    return data_list

In [5]:
def read_electricity_list(file_list, file_path):
    data_list = []
    data_list.append(format_eletricity_data(file_path+file_list[0], True))
    data_list.append(format_eletricity_data(file_path+file_list[1]))
    return data_list

In [6]:
def merge_data(climate_data, electricity_data):
    data = pd.merge(climate_data, electricity_data, on=['Date', 'Region ID'])
    data.drop(columns=[ 'Period end', 'Period start','Region ID', 'Region description', 'Region'], inplace=True)
        
    return data


### Read data files using helper functions above

In [7]:
ClimateDatafilelist = ['NewUpperNorthRegions.csv', 'NewLowerSouthRegions.csv', 'NewUpperSouthARegions.csv','NewLowerNorthRegions.csv', 'NewCentralNorthRegions.csv']
ClimatePath = '../ClimateDataProcessing/'
ElectricityDatafilelist = ['Business_demand_trends_20240901140306.csv', 'Residential_consumption_trends_20240901141207.csv']
ElectricityPath = '../ElectricityData/'

climate_data = read_climate_list(ClimateDatafilelist, ClimatePath)
electricity_data = read_electricity_list(ElectricityDatafilelist, ElectricityPath)

Remove the months not recorded

In [8]:
unobserved_rows = [175, 176, 177, 178, 179]
for i in range(len(climate_data)):
    climate_data[i].drop(unobserved_rows, inplace=True)

Find which data file has the missing value

In [9]:
for i in range(len(climate_data)):
    print(climate_data[i].isnull().sum()) 

Date                                                       0
Total Rainfall Mm                                          0
Mean Air Temperature in Celsius                            0
Mean Vapour Pressure Hpa                                   0
Mean Wind Speed M/Sec                                      0
Days Of Wind Gust >= 24 Knots in Day                       1
Standard Deviation Of Daily Mean Temperature in Celsius    0
Year                                                       0
Month                                                      0
Region ID                                                  0
dtype: int64
Date                                                       0
Total Rainfall Mm                                          0
Mean Air Temperature in Celsius                            0
Mean Vapour Pressure Hpa                                   0
Mean Wind Speed M/Sec                                      0
Days Of Wind Gust >= 24 Knots in Day                       0
Standard De

Check

In [10]:
climate_data[0][climate_data[0]['Days Of Wind Gust >= 24 Knots in Day'].isnull()]

,Date,Total Rainfall Mm,Mean Air Temperature in Celsius,Mean Vapour Pressure Hpa,Mean Wind Speed M/Sec,Days Of Wind Gust >= 24 Knots in Day,Standard Deviation Of Daily Mean Temperature in Celsius,Year,Month,Region ID
105,2018-10-01,59.65,14.55,13.2,3.4,NaN,1.45,2018.0,10.0,UNI


Fill by previous value

In [11]:
climate_data[0].ffill(inplace=True)

### Merge data files

Here we combine the dataframes to perform analysis by region easier.

In [12]:
electricity_Bus_res = pd.merge(electricity_data[0], electricity_data[1], how='inner')
electricity_Bus_res.head()


,Period start,Period end,Region ID,Region description,Selected business demand (GWh),Date,Region,Est. Total consumption (GWh)
0,1/01/10,31/01/10,CNI,Central North Island,502.788,2010-01-01,Central North Island,177.005
1,1/02/10,28/02/10,CNI,Central North Island,475.792,2010-02-01,Central North Island,162.780
2,1/03/10,31/03/10,CNI,Central North Island,539.230,2010-03-01,Central North Island,182.505
3,1/04/10,30/04/10,CNI,Central North Island,505.722,2010-04-01,Central North Island,188.663
4,1/05/10,31/05/10,CNI,Central North Island,519.263,2010-05-01,Central North Island,222.647


In [13]:
uni_climate_electricity = merge_data(climate_data[0], electricity_Bus_res)
cni_climate_electricity = merge_data(climate_data[4], electricity_Bus_res)
lni_climate_electricity = merge_data(climate_data[3], electricity_Bus_res)
usi_climate_electricity = merge_data(climate_data[2], electricity_Bus_res)
lsi_climate_electricity = merge_data(climate_data[1], electricity_Bus_res)

In [14]:
uni_climate_electricity.head()

,Date,Total Rainfall Mm,Mean Air Temperature in Celsius,Mean Vapour Pressure Hpa,Mean Wind Speed M/Sec,Days Of Wind Gust >= 24 Knots in Day,Standard Deviation Of Daily Mean Temperature in Celsius,Year,Month,Selected business demand (GWh),Est. Total consumption (GWh)
0,2010-01-01,41.9,20.05,17.40,4.20,11.0,1.70,2010.0,1.0,572.918,282.000
1,2010-02-01,34.8,21.35,19.70,3.65,8.0,1.20,2010.0,2.0,564.809,259.792
2,2010-03-01,7.6,19.30,16.65,4.35,12.5,1.45,2010.0,3.0,621.988,295.270
3,2010-04-01,48.4,17.05,16.05,3.15,7.0,1.40,2010.0,4.0,573.259,303.576
4,2010-05-01,197.5,14.60,14.40,3.55,12.5,1.95,2010.0,5.0,608.594,363.480


# External Data

External data we are adding are the population data which is yearly but also in regions. Import and export data are not regional however we can observe quaterly data.

TODO:

    POPULATION:
        map regions to zones
        map all years corresponding to the years. This is where the year feature is needed

    IMPORT EXPORT:
        map years and quaterly years to correct dates and also 
        aggregate for non months in years






### Import/ Export Data

In [15]:
imex = pd.read_csv('../ExternalData/hm8.csv')
imex.drop(index=range(0,4), inplace=True)                       # Remove first 4 rows (meta data of columns)

imex.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)       # Rename the column to Date             



In [16]:
# select the columns that are needed and ignore irrelevant columns
valid_columns = ['Date', 'Export prices', 'Export volumes', 'Export values', 'Import prices', 'Import volumes', 'Import values']
imex = imex[valid_columns]

# Select relevant rows and ignore irrelevant rows
valid_rows = range(83, max(imex.index)+1)
imex = imex.loc[valid_rows]


imex['Date'] = pd.to_datetime(imex['Date'], format='%b %Y')     # Convert the Date column to datetime format
imex['Date'] = imex['Date'] + pd.DateOffset(months=1)        # Offset the date by 1 month to match the electricity data 

imex.set_index('Date', inplace=True, drop=True)                # Set the Date column as the index
imex = imex.resample('ME').bfill()                            # Resample the data to monthly and backfill the missing values
imex.reset_index(inplace=True)                              

imex['Date'] = imex['Date'] - pd.offsets.MonthBegin(1)

c = imex.select_dtypes(object).columns
imex.replace(',','', regex=True, inplace=True)                # Remove commas from the data
imex[c] = imex[c].apply(pd.to_numeric, errors='coerce')            # Convert the data to numeric format


In [17]:
imex.dtypes

Date              datetime64[ns]
Export prices            float64
Export volumes           float64
Export values            float64
Import prices            float64
Import volumes           float64
Import values            float64
dtype: object

In [18]:
imex.head()

,Date,Export prices,Export volumes,Export values,Import prices,Import volumes,Import values
0,2010-01-01,1072.0,1160.0,10093.0,904.0,1360.0,8827.0
1,2010-02-01,1072.0,1160.0,10093.0,904.0,1360.0,8827.0
2,2010-03-01,1072.0,1160.0,10093.0,904.0,1360.0,8827.0
3,2010-04-01,1112.0,1264.0,11416.0,920.0,1476.0,9740.0
4,2010-05-01,1112.0,1264.0,11416.0,920.0,1476.0,9740.0


### Population Data

In [19]:
AnnualPopulation = pd.read_csv('../ExternalData/StatsNZ-subnational-population-estimates-2010-2023.csv')
AnnualPopulation=AnnualPopulation.dropna()          # Remove rows with missing values

# Format the Date column to datetime format
AnnualPopulation.loc[len(AnnualPopulation.index)] = [2024.0] + [None]*len(AnnualPopulation.columns[1:])
AnnualPopulation.loc[len(AnnualPopulation.index)] = [2025.0] + [None]*len(AnnualPopulation.columns[1:])
AnnualPopulation.rename(columns={'Estimated Resident Population for Regional Council Areas, at 30 June (1996+) (Annual-Jun)': 'Date'}, inplace=True)   
AnnualPopulation['Date'] = pd.to_datetime(AnnualPopulation['Date'], format='%Y')




In [20]:
UNI = ['Northland Region', 'Auckland Region']
CNI = ['Waikato Region', 'Bay of Plenty Region', 'Gisborne Region', 'Hawke\'s Bay Region']
LNI = ['Taranaki Region', 'Manawatu-Wanganui Region', 'Wellington Region']
USI = ['Tasman Region', 'Nelson Region', 'Marlborough Region', 'West Coast Region', 'Canterbury Region',]
LSI = ['Otago Region', 'Southland Region']
ZoneList = [UNI, LNI, USI, LSI, CNI]

In [21]:
population_data = []
for i in ZoneList:
    pd.set_option('future.no_silent_downcasting', True)
    df = pd.DataFrame(AnnualPopulation.loc[:,'Date'])
    df['Population'] = AnnualPopulation[i].sum(axis=1)

    df.replace(0, pd.NA, inplace=True)
    df.set_index('Date', inplace=True)
    df = df.resample('ME').ffill()
    df = df.ffill()
    df.reset_index(inplace=True)
    df['Date'] = df['Date'] - pd.offsets.MonthBegin(1)
    
    population_data.append(df)
    

In [22]:
population_data[0].head()

,Date,Population
0,2010-01-01,1600200.0
1,2010-02-01,1600200.0
2,2010-03-01,1600200.0
3,2010-04-01,1600200.0
4,2010-05-01,1600200.0


UNI_Population = pd.DataFrame((AnnualPopulation['Date']))
UNI_Population['Population'] = AnnualPopulation[UNI].sum(axis=1)
UNI_Population.replace(0, None, inplace=True)

UNI_Population.set_index('Date', inplace=True, drop=True)                # Set the Year column as the index
UNI_Population = UNI_Population.resample('ME').ffill()                      # Resample the data to monthly and backfill the missing values
UNI_Population.ffill().reset_index(inplace=True) 


# Merge Import and Population data with rest of data

In [23]:
imex.head()

,Date,Export prices,Export volumes,Export values,Import prices,Import volumes,Import values
0,2010-01-01,1072.0,1160.0,10093.0,904.0,1360.0,8827.0
1,2010-02-01,1072.0,1160.0,10093.0,904.0,1360.0,8827.0
2,2010-03-01,1072.0,1160.0,10093.0,904.0,1360.0,8827.0
3,2010-04-01,1112.0,1264.0,11416.0,920.0,1476.0,9740.0
4,2010-05-01,1112.0,1264.0,11416.0,920.0,1476.0,9740.0


In [24]:
uni_climate_electricity = uni_climate_electricity.merge(imex, on='Date')
cni_climate_electricity = cni_climate_electricity.merge(imex, on='Date')
lni_climate_electricity = lni_climate_electricity.merge(imex, on='Date')
usi_climate_electricity = usi_climate_electricity.merge(imex, on='Date')
lsi_climate_electricity = lsi_climate_electricity.merge(imex, on='Date')

uni_climate_electricity = uni_climate_electricity.merge(population_data[0], on='Date')
cni_climate_electricity = cni_climate_electricity.merge(population_data[4], on='Date')
lni_climate_electricity = lni_climate_electricity.merge(population_data[3], on='Date')
usi_climate_electricity = usi_climate_electricity.merge(population_data[2], on='Date')
lsi_climate_electricity = lsi_climate_electricity.merge(population_data[1], on='Date')


# Dealing with Outliers

In [25]:
uni_replacement_avg = (uni_climate_electricity.loc[uni_climate_electricity['Date'] == '2019-04-01', 'Selected business demand (GWh)'].values[0] + uni_climate_electricity.loc[uni_climate_electricity['Date'] == '2021-04-01', 'Selected business demand (GWh)'].values[0]) /2
cni_replacement_avg = (cni_climate_electricity.loc[cni_climate_electricity['Date'] == '2019-04-01', 'Selected business demand (GWh)'].values[0] + cni_climate_electricity.loc[uni_climate_electricity['Date'] == '2021-04-01', 'Selected business demand (GWh)'].values[0]) /2
lni_replacement_avg = (lni_climate_electricity.loc[lni_climate_electricity['Date'] == '2019-04-01', 'Selected business demand (GWh)'].values[0] + lni_climate_electricity.loc[lni_climate_electricity['Date'] == '2021-04-01', 'Selected business demand (GWh)'].values[0]) /2
usi_replacement_avg = (usi_climate_electricity.loc[usi_climate_electricity['Date'] == '2019-04-01', 'Selected business demand (GWh)'].values[0] + usi_climate_electricity.loc[usi_climate_electricity['Date'] == '2021-04-01', 'Selected business demand (GWh)'].values[0]) /2
lsi_replacement_avg = (lsi_climate_electricity.loc[lsi_climate_electricity['Date'] == '2019-04-01', 'Selected business demand (GWh)'].values[0] + lsi_climate_electricity.loc[lsi_climate_electricity['Date'] == '2021-04-01', 'Selected business demand (GWh)'].values[0]) /2

In [26]:
uni_climate_electricity.loc[uni_climate_electricity['Date'] == '2020-04-01', 'Selected business demand (GWh)'] = uni_replacement_avg
cni_climate_electricity.loc[cni_climate_electricity['Date'] == '2020-04-01', 'Selected business demand (GWh)'] = cni_replacement_avg
lni_climate_electricity.loc[lni_climate_electricity['Date'] == '2020-04-01', 'Selected business demand (GWh)'] = lni_replacement_avg
usi_climate_electricity.loc[usi_climate_electricity['Date'] == '2020-04-01', 'Selected business demand (GWh)'] = usi_replacement_avg
lsi_climate_electricity.loc[lsi_climate_electricity['Date'] == '2020-04-01', 'Selected business demand (GWh)'] = lsi_replacement_avg

# EXPORT DATA 

just click the code below and click 'Y' as the hotkey to convert markdown to code.


In [27]:
uni_climate_electricity.to_csv(f'./ProcessedData_2020-04_Adjusted/Electricity-ResBusClimateUNI.csv', index=False) 
cni_climate_electricity.to_csv(f'./ProcessedData_2020-04_Adjusted/Electricity-ResBusClimateCNI.csv', index=False)
lni_climate_electricity.to_csv(f'./ProcessedData_2020-04_Adjusted/Electricity-ResBusClimateLNI.csv', index=False)
usi_climate_electricity.to_csv(f'./ProcessedData_2020-04_Adjusted/Electricity-ResBusClimateUSI.csv', index=False)
lsi_climate_electricity.to_csv(f'./ProcessedData_2020-04_Adjusted/Electricity-ResBusClimateLSI.csv', index=False)